# Pangenome of colibactin containing Escherichia genomes

Construct pan and core genome of 36 colibactin containing Escherichia genomes using Roary software

In [ ]:
import os
import BCBio
from Bio import SeqIO
import pandas as pd
from shutil import copyfile
import pprint
import numpy as np
from Bio import Phylo
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

In [ ]:
# directories
roary_dir = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/'
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/genomes/'
adj_mat_dir = '/home/omkar/Projects/panGenome/data/entero_project/families/adj_mat/'
df_clusters_dir = '/home/omkar/Projects/panGenome/data/entero_project/general/df_clusters.p'
df_entero_dir = '/home/omkar/Projects/panGenome/data/entero_project/general/df_entero.p'

In [ ]:
subnet_id = 'comp_3'
pan_type = 'clusters' # or 'clusters'
tgt_genus = 'all'
mibig_id = 'BGC0000972.1'
mibig_name = 'clb'

In [ ]:
# main function
gff_dir = create_directories(roary_dir, subnet_id, pan_type, tgt_genus, mibig_name)
df_clusters, df_entero = get_saved_df(df_clusters_dir, df_entero_dir)
fam_tgt_clusters, df_fam_clusters, df_fam_genomes = get_fam_clusters(adj_mat_dir, subnet_id, mibig_id, tgt_genus, df_clusters, df_entero)
print('Number of target clusters:', len(fam_tgt_clusters))

copy_data_to_roary(data_dir, gff_dir, df_fam_clusters, pan_type)
# Run roary 
# Run on command line 
# > workon pangenome
# > cd /home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/
# > roary -f comp_3_clb_Escherichia/ comp_3_clb_Escherichia/gff/*.gff -v -r -e -n -i 90

In [ ]:
def create_directories(roary_dir, subnet_id, pan_type, tgt_genus, mibig_name):
    tgt_dir = os.path.join(roary_dir, pan_type, subnet_id + '_' + mibig_name + '_' + tgt_genus)
    if os.path.isdir(tgt_dir):
        print(tgt_dir , 'already exists!!')
    else:
        os.mkdir(tgt_dir)
    gff_dir = os.path.join(tgt_dir, 'gff')
    if os.path.isdir(gff_dir):
        print(gff_dir , 'already exists!!')
    else:
        os.mkdir(gff_dir)
    return gff_dir

def get_saved_df(df_clusters_dir, df_entero_dir):
    df_clusters = pd.read_pickle(df_clusters_dir)
    df_entero = pd.read_pickle(df_entero_dir)
    
    return df_clusters, df_entero

def get_fam_clusters(adj_mat_dir, subnet_id, mibig_id, tgt_genus, df_clusters, df_entero):
    df_input_dir = os.path.join(adj_mat_dir, subnet_id)
    df_adj_mat = pd.read_csv(df_input_dir)
    df_adj_mat = df_adj_mat.set_index(df_adj_mat.columns[0])
    df_adj_mat = df_adj_mat.iloc[:,2:-2]
    
    fam_clusters_list = []
    for col in df_adj_mat.columns:
        similarity = df_adj_mat.loc[mibig_id,col]
        if float(similarity) > 0.3:
            fam_clusters_list.append(col)
            
    idx = pd.IndexSlice

    if tgt_genus == 'all':
        df_fam_clusters = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,fam_clusters_list],:]
        fam_patric_ids = df_fam_clusters.index.get_level_values(4).tolist()
        fam_tgt_clusters = df_fam_clusters.index.get_level_values(11).tolist()
    else:
        df_fam_clusters = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,fam_clusters_list],:]
        df_fam_clusters = df_fam_clusters[df_fam_clusters.index.get_level_values(0) == tgt_genus]
        fam_patric_ids = df_fam_clusters.index.get_level_values(4).tolist()
        fam_tgt_clusters = df_fam_clusters.index.get_level_values(11).tolist()

    df_fam_genomes = df_entero.loc[fam_patric_ids,:]
    
    return fam_tgt_clusters, df_fam_clusters, df_fam_genomes 

def copy_data_to_roary(data_dir, gff_dir, df_fam_clusters, pan_type):
    fam_patric_ids = df_fam_clusters.index.get_level_values(4).tolist()
    fam_clusters_list = df_fam_clusters.index.get_level_values(11).tolist()
    
    if pan_type == 'genomes':
        for pat in fam_patric_ids:
            src = os.path.join(data_dir, pat, pat + '_roary.gff')
            dest = os.path.join(gff_dir, pat + '.gff')
            copyfile(src, dest)
            
    elif pan_type == 'clusters':
        for pat in fam_patric_ids:
            src = os.path.join(data_dir, pat, pat + '_roary.gff')
            dest = os.path.join(gff_dir, pat + '.gff')
            copyfile(src, dest)
        cluster_locusID_dict = dict()
        
        for tmp_id in range(len(fam_patric_ids)):
            pat = fam_patric_ids[tmp_id]
            clust = fam_clusters_list[tmp_id]
            gbk_path = os.path.join(data_dir, pat, 'as4', clust + '.gbk')
            cluster_locusID_dict[pat] = []
            with open(gbk_path, "r") as handle:
                for record in SeqIO.parse(handle, "genbank"):
                    for feature in record.features:
                        if feature.type == 'CDS':
                            locus_ID = feature.qualifiers['ID'][0]
                            cluster_locusID_dict[pat].append(locus_ID)
                            
        # Curate gff files by adding nucleotide sequence for Roary pangenome
        for tmp_id in range(len(fam_patric_ids)):
            pat = fam_patric_ids[tmp_id]
            clust = fam_clusters_list[tmp_id]
            
            in_file1 = os.path.join(gff_dir, pat + '.gff')
            out_file = os.path.join(gff_dir, clust + '.gff')

            with open(in_file1) as in_handle1, open(out_file, 'w') as out_handle:
                for line in in_handle1:
                    if line.startswith('accn'):
                        locus_ID = line.split('\t')[8].split(';')[0].split('=')[1]
                        if locus_ID in cluster_locusID_dict[pat]:
                            out_handle.write(line)
                    else:
                        out_handle.write(line)   
        for pat in fam_patric_ids:
            in_file1 = os.path.join(gff_dir, pat + '.gff')
            if os.path.exists(in_file1):
                os.remove(in_file1) 
                
                

In [ ]:
# Colibactin containing clusters : Comp 3 ecoli row BGC0000972.1 
adj_mat = pd.read_csv('/home/omkar/Projects/panGenome/data/entero_project/families/adj_mat/comp_3')
adj_mat = adj_mat.set_index(adj_mat.columns[0])
adj_mat = adj_mat.iloc[:,2:-2]
famX_clusters = []
for col in adj_mat.columns:
    similarity = adj_mat.loc['BGC0000972.1',col]
    if float(similarity) > 0.3:
        famX_clusters.append(col)

In [ ]:
df_clusters = pd.read_pickle('/home/omkar/Projects/panGenome/data/entero_project/general/df_clusters.p')
# df_ecoli = df_clusters[df_clusters]

In [ ]:
pan_ecoli_dir = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_CLB_MIBIG_escherichia/'
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/genomes/'
os.mkdir(pan_ecoli_dir)
# df_ecoli = pd.read_pickle('/home/omkar/Projects/panGenome/data/entero_project/general/df_ecoli.p')

In [ ]:
# Generate pangenome of E. coli strains with Colibactin and Yersiniabactin clusters - 30 (28 acc to BiG-SCAPE)
pat_clb_ybt_ecoli = df_ecoli.Strain_ID[df_ecoli.PKS_stat == 'Ybt-Clb'].tolist()

Ybt_Clb_dir = os.path.join(pan_ecoli_dir, 'Ybt_Clb')
os.mkdir(Ybt_Clb_dir)



In [ ]:
# Save genome metadata of Ybt-Clb strains of E coli 
df_clb = df_ecoli[df_ecoli.PKS_stat == 'Ybt-Clb']
df_clb= df_clb.loc[:,['Strain_ID', 'Genome_name', 'Contigs', 'Genome Length', 'GC Content', 'PATRIC CDS', 'RefSeq CDS', 'NCBI Taxon ID','GenBank Accessions', 'RefSeq Accessions', 'Strain', 'Serovar', 'Biovar', 'Pathovar', 'MLST','Publication']]
df_clb.set_index('Strain_ID', inplace=True)
df_clb.to_csv('/home/omkar/Projects/panGenome/data/entero_project/general/df_clb_ecoli.csv')

In [ ]:
gff_dir = os.path.join(Ybt_Clb_dir, 'gff')
os.mkdir(gff_dir)

In [ ]:
for pat in pat_clb_ybt_ecoli:
    src = os.path.join(data_dir, pat, pat + '_roary.gff')
    dest = os.path.join(gff_dir, pat + '.gff')
    copyfile(src, dest)

In [ ]:
# Run on command line 
# > workon pangenome
# > cd /home/omkar/Projects/panGenome/data/entero_project/pan_roary/ecoli/
# > roary -f Ybt_Clb/ Ybt_Clb/gff/*.gff -v -r

In [ ]:
# directories
roary_dir = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/'
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/genomes/'
df_clusters_dir = '/home/omkar/Projects/panGenome/data/entero_project/general/df_clusters.p'
df_entero_dir = '/home/omkar/Projects/panGenome/data/entero_project/general/df_entero.p'
homology_dir = '/home/omkar/Projects/panGenome/data/entero_project/homology/'
# variables 
subnet_id = 'comp_3'
pan_type = 'clusters' # or 'clusters'
tgt_genus = 'all'
mibig_id = 'BGC0000972.1'
mibig_name = 'clb'
core_cutoff = 1

# main function
roary_out = get_roary_out(roary_dir, pan_type, subnet_id, mibig_name, tgt_genus)
df_roary, df_roary_info, df_clusters, df_entero = get_saved_df(df_clusters_dir, df_entero_dir, roary_out)
df_roary_filtered, df_roary_bool, df_roary_sorted = filter_roary_pangenome(df_roary, core_cutoff)

get_pan_freq_plot(df_roary_sorted, roary_out)
get_pan_pie_chart(df_roary_sorted, roary_out)
get_core_pan_curve(roary_out)
get_presence_heatmap(df_roary_filtered, roary_out, df_entero)

# get_ecoli_genomes(df_entero)

# df_roary_filtered.columns = [df_entero.loc[pat_id, 'Strain'] for pat_id in df_roary_filtered.columns]

# sns.clustermap(df_roary_filtered, row_cluster=True, cmap = plt.cm.Blues, yticklabels = False, figsize=(15,20))
# df_entero_pks = df_entero.loc[df_roary.columns,:]

# nissle_genomes = ['316435.10', '316435.29']
# upec_genomes = df_entero_pks[df_entero_pks.Pathovar == 'UPEC'].index.tolist()
# expec_genomes = df_entero_pks[df_entero_pks.Pathovar == 'ExPEC'].index.tolist()
# aiec_genomes = df_entero_pks[df_entero_pks.Pathovar == 'AIEC'].index.tolist()

# df_nissle_absent = get_nissle_absent_genes(df_roary_bool, nissle_genomes, upec_genomes, expec_genomes, aiec_genomes)
# df_nissle_absent.columns = [df_entero.loc[pat_id, 'Strain'] for pat_id in df_nissle_absent.columns]
# nissle_absent_gene_list = df_nissle_absent.index.get_level_values('Gene')
# sns.set(font_scale=1.6)
# sns.clustermap(df_nissle_absent, cmap = plt.cm.Blues, yticklabels = nissle_absent_gene_list, figsize=(15,20))

# df_nissle_absent_features = get_nissle_absent_features(df_roary_info, '199310.4', df_nissle_absent, data_dir)
# homology_out_dir = create_homology_dir(homology_dir, mibig_name, tgt_genus)

# df_pan_features = pd.read_pickle(os.path.join(homology_out_dir, 'df_pan_features_unflitered.p'))
# df_interpro_all = pd.read

# run interpro on entire pangenome
# sudo ./interproscan.sh -i /home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_clb_Escherichia/_1553187017/pan_genome_reference.fa -b /home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_clb_Escherichia/inteproscan/ -goterms -pa


In [ ]:
df_roary_filtered = df_roary_sorted[df_roary_sorted.sum(1)>9]
# df_roary_sorted.sum(1)

In [ ]:
import pickle
df_clusters_clb = df_clusters[df_clusters.index.get_level_values(-1).isin(df_roary_filtered.columns)]
genus_list = df_clusters_clb.index.get_level_values(0)

pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/genus_colors.p',"rb")
my_palette = pickle.load(pickle_out)
pickle_out.close()
col_colors = genus_list.map(my_palette)

ax = sns.clustermap(df_roary_filtered, row_cluster=True, yticklabels = df_roary_filtered.index.get_level_values(0), col_cluster=True, cmap = 'Greens',figsize=(40,80), col_colors=col_colors)
# # ax2.cax.set_visible(False)

In [ ]:
ax2 = sns.clustermap(ax.data2d.iloc[:,:36], row_cluster=True, col_cluster=True, cmap = 'Greens',figsize=(40,80))

In [ ]:
# col_list = df_roary_bool.columns.tolist() 
# col_list[5] = 'Nissle 1917_2'

# df_roary_bool.columns = col_list

df_roary_block1 = df_roary_bool[df_roary_bool.sum(1)>=0.85*df_roary_bool.shape[1]]

df_roary_block2 = df_roary_bool[(df_roary_bool.sum(axis=1) < 0.85 * df_roary_bool.shape[1]) & (df_roary_bool.sum(axis=1) > 0.35*df_roary_bool.shape[1])]
ax2 = sns.clustermap(df_roary_block2, row_cluster=True, col_cluster=True, cmap = 'Greens', yticklabels = False, figsize=(13,21))
ax2.cax.set_visible(False)

roary_all_index = df_roary_block1.index.get_level_values(0).tolist() + ax2.data2d.index.get_level_values(0).tolist()
idx = pd.IndexSlice
df_roary_all = df_roary_bool.loc[idx[roary_all_index,:], ax2.data2d.columns]
cmap = [sns.color_palette('Greens')[0] , sns.color_palette('Greens')[-1]]
ax_all = sns.clustermap(df_roary_all, row_cluster=False, col_cluster=False, cmap = cmap, yticklabels = False, figsize=(16,21))
ax_all.cax.set_visible(False)
ax_all.ax_heatmap.set_ylabel('')
plt.savefig(os.path.join(roary_out, 'fig2_presence_heatmap_all.png'),pad_inches = 0.1, bbox_inches = 'tight')


In [ ]:
get_core_pan_curve(roary_out)

In [ ]:
df_roary_block2 = df_roary_bool[(df_roary_bool.sum(axis=1) < df_roary_bool.shape[1]) & (df_roary_bool.sum(axis=1) > 5)]
ax2 = sns.clustermap(df_roary_block2, row_cluster=True, col_cluster=True, cmap = 'Greens', yticklabels = True, figsize=(13,21))
ax2.cax.set_visible(False)

In [ ]:
core_path = os.path.join(roary_out,'number_of_conserved_genes.Rtab')
pan_path = os.path.join(roary_out,'number_of_genes_in_pan_genome.Rtab')
    
df_core_cnt = pd.read_table(core_path, sep='\t')
df_pan_cnt = pd.read_table(pan_path, sep='\t')
df_pan_cnt

print('absense: ',[255*col for col in sns.color_palette('Greens')[0]])
print('presense: ',[255*col for col in sns.color_palette('Greens')[-1]])

In [ ]:
# Import fasta file with pangenome reference
pan_ref_path = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_CLB_MIBIG/_1553084326/pan_genome_reference.fa'
pan_ref_filtered_path = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_CLB_MIBIG/pan_genome_filtered.fa'
pan_ref_filtered_trna_path = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_CLB_MIBIG/pan_genome_filtered.frn'


pan_filtered_genes = roary_filtered.index.get_level_values('Gene').tolist()
pat_strains_dict = dict()
gene_strains_dict = dict()

for record in SeqIO.parse(pan_ref_path, "fasta"):
    id_list = record.description.split(' ')
    gene = id_list[1]
    if gene in pan_filtered_genes:
        pat_gene_id = id_list[0]
        if 'peg' in pat_gene_id:
            tmp_list = pat_gene_id.split('.peg')
        elif  'rna' in pat_gene_id:
            tmp_list = pat_gene_id.split('.rna')
        else:
            print('Errror: Extra gene id :', pat_gene_id)
        pat_strain_id = tmp_list[0][4:]

        if pat_strain_id not in pat_strains_dict.keys():
            pat_strains_dict[pat_strain_id] = [pat_gene_id]
            gene_strains_dict[pat_gene_id] = gene
        else:
            pat_strains_dict[pat_strain_id].append(pat_gene_id)
            gene_strains_dict[pat_gene_id] = gene
    

In [ ]:
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/genomes/'
# Get protein fasta sequences for these genes to generate pangenome for homology analysis 
record_list = []
trna_list = []

for pat_strain_id in pat_strains_dict.keys():
    faa_path = os.path.join(data_dir, pat_strain_id, pat_strain_id + '.PATRIC.faa')
    rna_path = os.path.join(data_dir, pat_strain_id, pat_strain_id + '.PATRIC.frn')

    for record in SeqIO.parse(faa_path, "fasta"):
        id_list = record.id.split('|')
        pat_gene_id = 'fig|' + id_list[1]
        print(pat_gene_id)
        if pat_gene_id in pat_strains_dict[pat_strain_id]:
            record.description = record.description + ', ' + gene_strains_dict[pat_gene_id]
            record_list.append(record)
            
            
    for record in SeqIO.parse(rna_path, "fasta"):
        id_list = record.id.split('|')
        pat_gene_id = 'fig|' + id_list[1]
        print(pat_gene_id)
        if pat_gene_id in pat_strains_dict[pat_strain_id]:
            record.description = record.description + ', ' + gene_strains_dict[pat_gene_id]
            trna_list.append(record)  
            
SeqIO.write(record_list, pan_ref_filtered_path, "fasta")
SeqIO.write(trna_list, pan_ref_filtered_trna_path, "fasta")

### Write gene features file for pangenome

In [ ]:
# Load roary file again
Ybt_Clb_path = '/home/omkar/Projects/panGenome/data/entero_project/pan_roary/genomes/comp_3_CLB_MIBIG/_1553084326/gene_presence_absence.csv'
roary = pd.read_table(Ybt_Clb_path,
                     sep=',',
                     low_memory=False)
roary.set_index(['Gene', 'Annotation'], inplace=True)

# Drop the other info columns
roary.drop(list(roary.columns[:12]), axis=1, inplace=True)

In [ ]:
# Get features dataframes
# Only for CDS at the moment, include RNAs later

CFT_features_path = os.path.join(data_dir, '199310.4', '199310.4.PATRIC.features.tab') 
df_CFT_features = pd.read_csv(CFT_features_path, sep='\t')
df_CFT_features = df_CFT_features[df_CFT_features.feature_type == 'CDS']
df_CFT_features = df_CFT_features.loc[:,['genome_name', 'patric_id', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']]
df_CFT_features = df_CFT_features.set_index('patric_id')

IHE_features_path = os.path.join(data_dir, '714962.3', '714962.3.PATRIC.features.tab') 
df_IHE_features = pd.read_csv(IHE_features_path, sep='\t')
df_IHE_features = df_IHE_features[df_IHE_features.feature_type == 'CDS']
df_IHE_features = df_IHE_features.loc[:,['genome_name', 'patric_id', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']]
df_IHE_features = df_IHE_features.set_index('patric_id')

Nissle_features_path = os.path.join(data_dir, '316435.10', '316435.10.PATRIC.features.tab') 
df_Nissle_features = pd.read_csv(Nissle_features_path, sep='\t')
df_Nissle_features = df_Nissle_features[df_Nissle_features.feature_type == 'CDS']
df_Nissle_features = df_Nissle_features.loc[:,['genome_name', 'patric_id', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']]
df_Nissle_features = df_Nissle_features.set_index('patric_id')

In [ ]:
# Import fasta file with pangenome reference
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/genomes/'

# Get .PATRIC.features for these genes to generate features of pangenome for homology analysis 
# Priority given to CFT073, IHE3034, Nissle 1917 and then whatever is chosen in pan_fasta file features
df_pan_features = pd.DataFrame(columns=['genome_name', 'pan_gene_id','refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']) 

pan_filtered_genes = roary.index.get_level_values('Gene').tolist()
pat_strains_dict = dict()
gene_strains_dict = dict()


for record in SeqIO.parse(pan_ref_path, "fasta"):
    id_list = record.description.split(' ')
    gene = id_list[1]
    pat_gene_id = id_list[0]
        
    if gene in pan_filtered_genes:
        if not roary.loc[gene,'199310.4'].isnull().any().any():
            CFT_gene_id = roary.loc[gene,'199310.4'].values[0]
            
            if CFT_gene_id in df_CFT_features.index:
                df_pan_features.loc[CFT_gene_id,:] = df_CFT_features.loc[CFT_gene_id, :]
                df_pan_features.loc[CFT_gene_id,'pan_gene_id'] = gene
                df_pan_features.loc[CFT_gene_id,'pan_pat_id'] = pat_gene_id
            else:
                print('1', gene)
                
        elif not roary.loc[gene,'714962.3'].isnull().any().any():
            IHE_gene_id = roary.loc[gene,'714962.3'].values[0]
            
            if IHE_gene_id in df_IHE_features.index:
                df_pan_features.loc[IHE_gene_id,:] = df_IHE_features.loc[IHE_gene_id, :]
                df_pan_features.loc[IHE_gene_id,'pan_gene_id'] = gene
                df_pan_features.loc[IHE_gene_id,'pan_pat_id'] = pat_gene_id
            else:
                print('2', gene)            
            
        elif not roary.loc[gene,'316435.10'].isnull().any().any():
            Nissle_gene_id = roary.loc[gene,'316435.10'].values[0]
            
            if Nissle_gene_id in df_Nissle_features.index:
                df_pan_features.loc[Nissle_gene_id,:] = df_Nissle_features.loc[Nissle_gene_id, :]
                df_pan_features.loc[Nissle_gene_id,'pan_gene_id'] = gene
                df_pan_features.loc[Nissle_gene_id,'pan_pat_id'] = pat_gene_id
            else:
                print('4', gene)
            
                
        else:
            if 'peg' in pat_gene_id:
                tmp_list = pat_gene_id.split('.peg')
                pat_strain_id = tmp_list[0][4:]
                
                other_features_path = os.path.join(data_dir, pat_strain_id, pat_strain_id + '.PATRIC.features.tab') 
                
                df_other_features = pd.read_csv(other_features_path, sep='\t')
                df_other_features = df_other_features[df_other_features.feature_type == 'CDS']
                df_other_features = df_other_features.loc[:,['patric_id', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 
                                'gene', 'product', 'figfam_id', 'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']]
                df_other_features = df_other_features.set_index('patric_id')
                
                if pat_gene_id in df_other_features.index:
                    df_pan_features.loc[pat_gene_id,:] = df_other_features.loc[pat_gene_id, :]
                    df_pan_features.loc[pat_gene_id,'pan_gene_id'] = gene
                    df_pan_features.loc[pat_gene_id,'pan_pat_id'] = pat_gene_id

In [ ]:
bigscape_dir = '/home/omkar/Projects/panGenome/data/entero_project/bigscape_specific_inputs/comp_3_clb_all/'
clusters_tmp_list = os.listdir(bigscape_dir)
for cluster_id in clusters_tmp_list:
    if cluster_id[:-4] not in fam_tgt_clusters:
        print(cluster_id)
    else:
        print('ok')

In [ ]:
df_pan_features.to_pickle('/home/omkar/Projects/panGenome/data/entero_project/homology/pan_clb_cluster/homology_maps/pan_features_df.p')
df_pan_features.to_csv('/home/omkar/Projects/panGenome/data/entero_project/homology/pan_clb_cluster/homology_maps/pan_features_df.csv')

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

import os
import BCBio
from Bio import SeqIO
import pandas as pd
from shutil import copyfile
import pprint
import numpy as np
from Bio import Phylo
from scipy.cluster.hierarchy import linkage

def get_core_pan_curve(roary_out):
    core_path = os.path.join(roary_out,'number_of_conserved_genes.Rtab')
    pan_path = os.path.join(roary_out,'number_of_genes_in_pan_genome.Rtab')
    
    df_core_cnt = pd.read_table(core_path, sep='\t')
    df_pan_cnt = pd.read_table(pan_path, sep='\t')
    
    avg_core = df_core_cnt.sum(0)/df_core_cnt.shape[0]
    max_core = df_core_cnt.max(0)
    min_core = df_core_cnt.min(0)
    
    avg_pan = df_pan_cnt.sum(0)/df_core_cnt.shape[0]
    max_pan = df_pan_cnt.max(0)
    min_pan = df_pan_cnt.min(0)
    
    genome_numbers = list(range(1,df_core_cnt.shape[1]+1))
    
    plt.figure(figsize=(8,8))
    
    plt.plot(genome_numbers, avg_core, color='green', linewidth=4)
    plt.plot(genome_numbers, avg_pan, color='lightgreen', linewidth=4)
    
    
    plt.plot(genome_numbers, min_core, color='green', linestyle='dashed', linewidth=4)
    plt.plot(genome_numbers, max_core, color='green', linestyle='dashed', linewidth=4)
    
    plt.plot(genome_numbers, min_pan,  color='lightgreen',linestyle='dashed', linewidth=4)
    plt.plot(genome_numbers, max_pan, color='lightgreen', linestyle='dashed', linewidth=4)
    
    plt.fill_between(genome_numbers, avg_core, y2=0, color = sns.color_palette('Greens')[3])
    plt.fill_between(genome_numbers, avg_pan, y2=avg_core, color = sns.color_palette('Greens')[1])
    
    plt.ylim(ymin=0)
    plt.xlim(xmin=1,xmax=36)
    plt.xlabel('No. of genomes')
    plt.ylabel('No. of gene families')
    
#     legend([p2, p1], ["line 2", "line 1"])
    
    plt.legend(['Core genome', 'Pan genome'])
    
    plt.savefig(os.path.join(roary_out, 'fig3_pan_core_curves.png'))
    
    


def get_nissle_absent_features(df_roary_info, CFT_pat, df_nissle_absent, data_dir):
    CFT_gene_id_list = df_roary_info.loc[df_nissle_absent.index, CFT_pat].tolist()
    
    CFT_features_path = os.path.join(data_dir, CFT_pat, CFT_pat + '.PATRIC.features.tab') 
    df_CFT_features = pd.read_csv(CFT_features_path, sep='\t')
    df_CFT_features = df_CFT_features[df_CFT_features.feature_type == 'CDS']
    df_CFT_features = df_CFT_features.loc[:,['genome_name', 'patric_id', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway', 'CFT_gene_id']]
    df_CFT_features = df_CFT_features.set_index('patric_id')
       
        
    df_nissle_absent_features = pd.DataFrame(columns=['genome_name', 'refseq_locus_tag', 'alt_locus_tag', 'uniprotkb_accession', 'gene', 'product', 'figfam_id',
       'plfam_id', 'pgfam_id', 'go', 'ec', 'pathway']) 
    temp_cnt = 0
    for CFT_gene_id in CFT_gene_id_list:
        pan_gene_id = df_nissle_absent.index.get_level_values('Gene')[temp_cnt]
        if CFT_gene_id in df_CFT_features.index:
            df_nissle_absent_features.loc[pan_gene_id,'CFT_gene_id'] = CFT_gene_id
            df_nissle_absent_features.loc[pan_gene_id,:] = df_CFT_features.loc[CFT_gene_id, :]
        temp_cnt = temp_cnt + 1    
            
    return df_nissle_absent_features

def get_nissle_absent_genes(df_roary_bool, nissle_genomes, upec_genomes, expec_genomes, aiec_genomes):
    df_nissle_absent = df_roary_bool[(df_roary_bool.sum(axis=1) < df_roary_bool.shape[1]) & 
                         (df_roary_bool[nissle_genomes[0]] == 0) & 
                         (df_roary_bool[nissle_genomes[1]] == 0) & 
                         (df_roary_bool[upec_genomes[0]] == 1) & 
                         (df_roary_bool[upec_genomes[1]] == 1) & 
                         (df_roary_bool[upec_genomes[2]] == 1) & 
                         (df_roary_bool[upec_genomes[3]] == 1) & 
                         (df_roary_bool[upec_genomes[4]] == 1) & 
                         (df_roary_bool[upec_genomes[5]] == 1) & 
                         (df_roary_bool[expec_genomes[0]] == 1) &
                         (df_roary_bool[aiec_genomes[0]] == 1)]
    
    df_nissle_absent = df_nissle_absent.drop(columns=nissle_genomes)
    
    return df_nissle_absent
    
def create_homology_dir(homology_dir, mibig_name, tgt_genus):
    tmp_dir = os.path.join(homology_dir, 'pan_' + mibig_name + '_vs_' + tgt_genus)
    if os.path.isdir(tmp_dir):
        print(tmp_dir, 'already exists!')
    else:
        os.mkdir(tmp_dir)
        
    tgt_dir = os.path.join(tmp_dir, 'tgt_faa')
    if os.path.isdir(tgt_dir):
        print(tgt_dir, 'already exists!')
    else:
        os.mkdir(tgt_dir)
        
    temp_dir = os.path.join(tmp_dir, 'temp_faa')
    if os.path.isdir(temp_dir):
        print(tgt_dir, 'already exists!')
    else:
        os.mkdir(temp_dir)
    return tmp_dir

def filter_roary_pangenome(df_roary, core_cutoff):
    df_roary_bool = df_roary.replace('.{2,100}', 1, regex=True)
    df_roary_bool.replace(np.nan, 0, regex=True, inplace=True)    
    df_roary_filtered = df_roary_bool[(df_roary_bool.sum(axis=1) < df_roary_bool.shape[1]) & (df_roary_bool.sum(axis=1) > 1)]
    
    # Sort the matrix by the sum of strains presence
    idx = df_roary_bool.sum(axis=1).sort_values(ascending=False).index
    df_roary_sorted = df_roary_bool.ix[idx]
    
    return df_roary_filtered, df_roary_bool, df_roary_sorted


def get_roary_out(roary_dir, pan_type, subnet_id, mibig_name, tgt_genus):
    roary_tmp = os.path.join(roary_dir, pan_type, subnet_id + '_' + mibig_name + '_' + tgt_genus)
    file_list = [file for file in os.listdir(roary_tmp) if '_' in file]
    print(file_list)
    if len(file_list) > 1:
        print('Too many output pangenomes!!')
        return None
    else:
        roary_out = os.path.join(roary_tmp, file_list[0])
    return roary_out

def get_saved_df(df_clusters_dir, df_entero_dir, roary_out):
    df_clusters = pd.read_pickle(df_clusters_dir)
    df_entero = pd.read_pickle(df_entero_dir)
    
    gene_presence_path = os.path.join(roary_out, 'gene_presence_absence.csv')
    df_roary = pd.read_table(gene_presence_path, sep=',', low_memory=False)
    # Set index (group name)
    df_roary.set_index(['Gene', 'Annotation'], inplace=True)
    df_roary_info = df_roary.copy(deep = True)
    
    # Drop the other info columns
    df_roary = df_roary.drop(list(df_roary.columns[:12]), axis=1)
    return df_roary, df_roary_info, df_clusters, df_entero

def get_pan_freq_plot(roary, roary_out):
    plt.figure(figsize=(10, 7))
    plt.hist(roary.sum(axis=1), roary.shape[1], color='purple',
         histtype="stepfilled", alpha=.7)
    plt.xlabel('Number of genomes', fontweight = 'bold', fontsize = '20')
    plt.ylabel('Number of genes', fontweight = 'bold', fontsize = '20')
    plt.xticks(fontsize = '18')
    plt.yticks(fontsize = '18')
    
    plt.savefig(os.path.join(roary_out, 'fig1_pan_freq.png'),pad_inches = 0.1, bbox_inches = 'tight')
    
def get_pan_pie_chart(roary, roary_out):
    # Plot the pangenome pie chart
    plt.figure(figsize=(10, 10))

    core = roary[roary.sum(axis=1) == roary.shape[1]].shape[0]
    softcore = roary[(roary.sum(axis=1) < roary.shape[1]) &
                     (roary.sum(axis=1) >= roary.shape[1]*0.85)].shape[0]
    shell = roary[(roary.sum(axis=1) < roary.shape[1]*0.85) &
                     (roary.sum(axis=1) >= roary.shape[1]*0.15)].shape[0]
    cloud = roary[roary.sum(axis=1) < roary.shape[1]*0.15].shape[0]

    total = roary.shape[0]
    
    def my_autopct(pct):
        val=int(pct*total/100.0)
        return '{v:d}'.format(v=val)
    a=plt.pie([core, softcore, shell, cloud],
      labels=['core  (%d strains)'%roary.shape[1],
              'soft-core  (%d <= strains < %d)'%(roary.shape[1]*.85,
                                                 roary.shape[1]),
              'shell\n(%d <= strains < %d)'%(roary.shape[1]*.15,
                                             roary.shape[1]*.85),
              'cloud\n(strains < %d)'%(roary.shape[1]*.15)],
      explode=[0.1, 0.05, 0.02, 0], radius=0.9,
      colors=[(0, 0, 1, float(x)/total) for x in (core, softcore, shell, cloud)],
      autopct=my_autopct, textprops={'fontsize': 20, 'fontweight': 'bold'})
    
    plt.savefig(os.path.join(roary_out, 'fig2_pan_pie.png'),pad_inches = 0.1, bbox_inches = 'tight')
    
def get_presence_heatmap(roary, roary_out, df_entero):
#     roary.columns = [df_entero.loc[pat_id, 'Strain'] for pat_id in roary.columns]
    ax = sns.clustermap(roary, row_cluster=True, col_cluster=True, cmap = 'Greens', yticklabels = False, figsize=(13,21))
    ax.cax.set_visible(False)
    
    plt.savefig(os.path.join(roary_out, 'fig2_presence_heatmap.png'),pad_inches = 0.1, bbox_inches = 'tight')
    
